Following Free Code Camps tutorial closely.

In [1]:
import ollama
import numpy as np
import torch

In [2]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.2")

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Lecture 1 Magnetic fields and magnetic force.pdf")
#loader2 = PyPDFLoader("Lecture 7- The MOS transistors.pdf")
docs = loader.load()
#docs2 = loader2.load()

print(f"Total characters: {len(docs[2].page_content)}")
#print(f"Total characters: {len(docs2[5].page_content)}")

Total characters: 1647


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#all_docs = docs + docs2

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 512,
    chunk_overlap = 64
)

#Make splits
splits = text_splitter.split_documents(docs)

In [7]:
print(docs[0].page_content)

PHYS2003: Physics for Electrical Engineers, semester 1 2023 
Lecture 1 
1 
 
Lecture 1: Magnetic fields and magnetic force 
Learning outcomes 
Assessable 
• Calculate the force on moving charges in a magnetic field via the cross product. 
• Calculate the direction of resulting vectors from the cross product or, where 
appropriate, apply basic rules to determine the direction. 
• Calculate potential difference in a moving conductor. 
Understanding 
• Visualize and conceptually explain the motion of charged particles in magnetic fields. 
1. Summary 
1.1. Purpose of PHYS2003 
The purpose of PHYS2003 is to provide you with the fundamental physics basis of 
phenomena and devices that you will draw on as you advance in your studies. In particular, 
units such as ELEC3016 (Power and Machines), ELEC3014 (Electronic Materials and Devices) 
and ELEC4407 (Engineering Electromagnetics) will draw heavily on the understanding of 
electromagnetism and quantum mechanics that you develop in PHYS2003. 


In [8]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=splits,  # these are already LangChain `Document` objects
    embedding=embeddings,
    collection_name="circuit_docs",
    persist_directory="./chroma_db"
)

retriever = vectorstore.as_retriever(search_kwargs={'k': 10})

From RAG Tutorial this appears to end the embedding and indexing section, next video we will see retrieval

# Part3 - Retrieval

In this section code camp goes over the retrieval section

In [9]:
docsearch = retriever.invoke("Edwin H. Hall experiment")

In [10]:
print(docsearch)

[Document(id='2e03c74d-0bf1-4952-a642-5e965c694e5e', metadata={'source': 'Lecture 1 Magnetic fields and magnetic force.pdf', 'page': 5, 'producer': 'macOS Version 12.5 (Build 21G72) Quartz PDFContext, AppendMode 1.1', 'author': 'David Gozzard', 'moddate': "D:20230305080614Z00'00'", 'creationdate': "D:20230224061506Z00'00'", 'creator': 'Microsoft® Word 2016', 'page_label': '6', 'total_pages': 7}, page_content='conductor are positive or negative, and measure the drift speed of electrons (if we move the \nstrip through the magnetic field with same speed as the electron drift, but in the opposite \ndirection, the Hall Effect voltage will be zero), we can create a device where the output \nvoltage is proportional to the strength of the magnetic field. Such a Hall effect sensor (or \nHall sensor) has many applications. \nHall effect sensors can detect both the magnitude and orientation of a magnetic field, and \nthey can work with static (unchanging) magnetic fields. They have no moving part

In [11]:
for i, doc in enumerate(docsearch):
    print(f"\n--- Document {i+1} ---\n")
    print(doc.page_content)


--- Document 1 ---

conductor are positive or negative, and measure the drift speed of electrons (if we move the 
strip through the magnetic field with same speed as the electron drift, but in the opposite 
direction, the Hall Effect voltage will be zero), we can create a device where the output 
voltage is proportional to the strength of the magnetic field. Such a Hall effect sensor (or 
Hall sensor) has many applications. 
Hall effect sensors can detect both the magnitude and orientation of a magnetic field, and 
they can work with static (unchanging) magnetic fields. They have no moving parts, so are 
not prone to wear, and can operate at much higher speeds than mechanical switches. Hall

--- Document 2 ---

PHYS2003: Physics for Electrical Engineers, semester 1 2023 
Lecture 1 
6 
 
 
Figure 3: Simplified diagram of the J. J. Thomson experiment that discovered the electron. 
5.2. The Hall Effect 
In a conductor, when there is no current through it, the electrons move randomly with

# Part 4 - Generation

In [ ]:
from langchain.prompts import PromptTemplate

template = """Answer the following question only using the following context:
{context}

If the answer is not contained in the context, respond with:
"I cannot answer this question because the necessary information was not found in the provided documents."

When answering, include the **source file name** and **slide/page number** if available.

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt

In [ ]:
llm = OllamaLLM(model = "llama3.2")

In [ ]:
chain = prompt | llm

In [ ]:
print(chain.invoke({"context":docsearch, "question":"Describe an experiment used to find the Hall effect"}))